# Introduction to metaprogramming

Normal programs manipulate data. Metaprograms **manipulate code**. (In many languages this is impossible.)
Of course, the ultimate goal is that the programs that we manipulate will later operate on data.

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "programming a program"

i.e. writing code (a program) to manipulate not data, but code. (And this code will, itself, then manipulate data [or another program...].)

Conjecture: metaprogramming, not taught much in scientific computing, is what we need more of as optimization, autodiff, and dimension reduction is not operating on whole programs.

## Symbols and Expressions

In [ ]:
:a, :b, :c, :foo, :α, :call # These are all symbols

In [ ]:
:(a+1),:(sin(b)),:(c(10)),:(foo(a,b,c)),:(foo(α^2+1)) # These are all expressions

A really good explanation of the what and why of symbols in Julia: [Stack Overflow](https://stackoverflow.com/questions/23480722/what-is-a-symbol-in-julia)

We use the special syntax `:( ... )` to represent a piece of Julia code.  It is called quoting.

In [ ]:
ex = :(x+3)

We can see that the object `ex` is composied of objects and symbols:

In [ ]:
dump(ex)

In [ ]:
typeof(ex)

We see that `ex` is of type `Expr`, meaning a Julia expression object - a piece of Julia code, stored as an object inside Julia itself!

The details about the object are stored (as usual) in its fields. The kind of expression object is stored as the `head` field:

In [ ]:
ex.head

In this case, we see that it represents a function call. 

The arguments of the function call are stored in the `ex.args` field:

In [ ]:
ex.args

In [ ]:
ex.args[2]

We can now *manipulate* and *modify* the object by *changing* these fields:

In [ ]:
ex.args[2] = :z
ex.args[3] = :4

In [ ]:
ex

We see that the expression has changed. We can even change it from an addition to something else:

In [ ]:
ex.args[1] = :*

In [ ]:
ex

This is the basis of metaprogramming, since it gives us the ability to take one piece of code, and modify it to produce another piece of code.

The foundation of an expression are `Symbol`s, written with initial colons (`:`):


In [ ]:
typeof(:z)

A more compact form to display an expression is as follows:

In [51]:
using Pkg

In [53]:
Pkg.add("MacroTools")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /Users/edelman/.julia/registries/General — failed to fetch from repo
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:144


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [1914dd2f] + MacroTools v0.4.4
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [48]:
using MacroTools

ArgumentError: ArgumentError: Package MacroTools not found in current path:
- Run `Pkg.add("MacroTools")` to install the MacroTools package.


In [ ]:
Meta.show_sexpr(ex)

## More complicated expressions

What about more complicated expressions?

In [ ]:
ex2 = :(x + 3y)

In [ ]:
Meta.show_sexpr(ex2)

In [ ]:
dump(ex2)

In [ ]:
ex2.args[3]

In [ ]:
typeof(ex2.args[3])

We see that expressions are recursive, in the sense that the subpiece `:(3y)` is itself another `Expr` object.

Julia provides a mechanism to create an `Expr` from a `String`, via the `parse` function:

In [ ]:
Meta.parse("x + 3y")

In [ ]:
dump(:(x+3x^2))

# Build Your Own

## Manipulating expressions

How can we do something more complicated? For example, suppose we start with the expression

In [ ]:
ex = :(x + 3x^2)

In [ ]:
dump(ex)

and we would like to change *all* `x`s to `(x+1)`s? We will need to dig down into the structure.
A first attempt is the following function. Note that this *modifies* its argument, so has a `!` ("bang") in the function name.

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        
        if args[i] == :x
            args[i] = :(x+1)
        end
        
    end
end

In [ ]:
traverse!(ex)

In [ ]:
ex

Oops, that didn't work -- we didn't manage to reach inside the inner `Expr` object. 

In [ ]:
ex = :(x + 3x^2)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        if args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [ ]:
traverse!(ex)

The AST ("Abstract Syntax Tree") has a **recursive** structure, so we use a recursive function:

In [ ]:
Meta.show_sexpr(ex)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        
        if isa(args[i], Expr)
            traverse!(args[i])
            
        elseif args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [33]:
traverse(ex) = ex  # generally do nothing
traverse(ex::Symbol) = ex==:x ? :(x+1) : ex # for the symbol :x make it :(x+1)
traverse(ex::Expr) = Expr(:call,traverse.(ex.args)...) # Expressions are done recursively

traverse (generic function with 3 methods)

In [34]:
ex = :(x + 3x^2)
traverse(ex)

:((x + 1) + 3 * (x + 1) ^ 2)

**Exercise**: Make the `traverse!` function more general, to replace `x` by an arbitrary expression that is another argument of the function.

## Evaluation

Now that we have our new object, we want to use it in Julia. We do this using `eval`:


In [85]:
eval(x) = Core.eval(Main, x)

eval (generic function with 1 method)

In [88]:
ex1 = :(3x^2)
x=10
eval(ex1)

:(3 * x ^ 2)

In [90]:
op = :^
ex2 = :(3*$op(x,2))
eval(ex2)

300

In [83]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Int64 3
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol x
        3: Int64 2


In [84]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Int64 3
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol x
        3: Int64 2


In [38]:
x = 3

3

In [39]:
eval(ex)

27

In [80]:
op = :^
ex = :(3*$op(x,2))
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Int64 3
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ^
        2: Symbol x
        3: Int64 2


## Using multiple dispatch with expressions

What if, say, we want to wrap "number literals" like 3, 4.5 with `f(3)` etc.?
We will write a more Julian version, using **multiple dispatch**:

In [41]:
function wrap_literals(ex::Expr, f::Symbol)
    
    #println("Entering !")
    #@show ex, s, new_expr
    
    args = ex.args
    
    for i in 1:length(args)
        #@show i, args[i]
        args[i] = wrap_literals(args[i], f)
        
    end
    
    return ex
end

wrap_literals(x::Number, f::Symbol) = :($f($x))
wrap_literals(x, f) = x  # fall-back method

wrap_literals (generic function with 3 methods)

In [42]:
ex = :(3x^2)
wrap_literals(:(3x^2), :f)

:(f(3) * x ^ f(2))

## Inserting pieces of code

Wilkinson-type polynomials are polynomials like

$p_5(x) = (x-1) (x-2) (x-3) (x-4) (x-5)$

In Julia we could write

In [43]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

p_5 (generic function with 1 method)

**Exercise:** We can interpolate pieces of code in other pieces using the `$` operator, and symbols from strings using `Symbol`. Make and evaluate an expression to define $p_n$ for different $n$, and interact with it.

## Generating repetitive code

One of the uses of metaprogramming is **code generation**. 
Suppose that for some reason we want to make a type that behaves like a `Float64`, but with some additional functionality.:

In [44]:
struct MyFloat
    a::Float64
end

We would need to write methods like


In [45]:
import Base: +, -

+(x::MyFloat, y::MyFloat) = x.a + y.a
-(x::MyFloat, y::MyFloat) = x.a - y.a

- (generic function with 176 methods)

This will get boring fast. Whenever we have repetition like this, we should get the computer to do it for us. However, here we need to **repeat code**. Let's make a code *template*; it should look like

    op(x::MyFloat, y::MyFloat) = op(x.a, y.a)

We want a code object that looks like that:

In [55]:
ex =(:(op(x::MyFloat, y::MyFloat) = op(x.a, y.a)))

:(op(x::MyFloat, y::MyFloat) = begin
          #= In[55]:1 =#
          op(x.a, y.a)
      end)

We can also write this as 

In [56]:
ex = quote
        op(x::MyFloat, y::MyFloat) = op(x.a, y.a)
    end 

quote
    #= In[56]:2 =#
    op(x::MyFloat, y::MyFloat) = begin
            #= In[56]:2 =#
            op(x.a, y.a)
        end
end

However, we want to replace `op` **by its value**

In [58]:
op = :+
ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end 

quote
    #= In[58]:3 =#
    x::MyFloat + y::MyFloat = begin
            #= In[58]:3 =#
            x.a + y.a
        end
end

And now just loop:

In [59]:
import Base: +,-,*,/

for op in (:+, :-, :*, :/)
    
    ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end
    
    eval(ex)
end


Julia provides `@eval` to make this shorter (by not needing the quote):

In [92]:
for op in (:+, :-, :*, :/)
    
    @eval $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    
end


In [114]:
 x=1
 y=1
 op = :+
 @eval  a=$op(x, y) 
 a

2

In [97]:
op = :^
ex = :(3*$op(x,2))
eval(ex)
eval(:(3*$op(x,2)))

300

## Macros

We don't want the user to have to think about Julia syntax objects. We can supply a **macro**.
This can be thought of as a "meta-function": it takes a Julia expression object as argument, and returns a Julia expression object. However, when it is called, it *automatically parses its input into the expression object*.

The simplest example:

In [145]:
macro simple(ex)
    println(ex)
    println(typeof(ex))
    show(dump(ex))
    return ex
end
    

@simple (macro with 1 method)

In [146]:
@simple a=[1,2,3]

a = [1, 2, 3]
Expr
Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol a
    2: Expr
      head: Symbol vect
      args: Array{Any}((3,))
        1: Int64 1
        2: Int64 2
        3: Int64 3
nothing

3-element Array{Int64,1}:
 1
 2
 3

We can now make a macro `wrap` that calls our function:

In [147]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    show(new_ex)
end

@wrap (macro with 1 method)

In [148]:
@wrap g 3x+2

:(g(3) * x + g(2))

We can also call it as `@wrap(g, 3x+2)`:

In [149]:
@wrap(g, 3x+2)

:(g(3) * x + g(2))

In fact, macros should **return an expression**. This expression will be **evaluated in the scope in which it is called**:

In [150]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    return new_ex
end

@wrap (macro with 1 method)

In [151]:
@wrap(g, 3x+2)

UndefVarError: UndefVarError: g not defined

In [152]:
x = 3
g(x) = 10x

g (generic function with 1 method)

In [153]:
@wrap g 3x+2

110

In [154]:
g(3)*x+g(2)

110

In [157]:
macro playwith(e1,e2)
    show(dump(e1))
    show(dump(e2))
    return(e1)
end

@playwith (macro with 1 method)

In [158]:
@playwith 1+1 2+2

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
nothingExpr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 2
    3: Int64 2
nothing

2

In [165]:
a = rand(3)'
dump(a)

LinearAlgebra.Adjoint{Float64,Array{Float64,1}}
  parent: Array{Float64}((3,)) [0.30876, 0.665151, 0.650055]


In [168]:
supertype(typeof(a))

AbstractArray{Float64,2}

In [182]:
a = rand(3)'
? sum

LoadError: syntax: invalid identifier name "?"

In [186]:
?sum

search: sum sum! summary cumsum cumsum! isnumeric VersionNumber issubnormal



```
sum(f, itr)
```

Sum the results of calling function `f` on each element of `itr`.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

# Examples

```jldoctest
julia> sum(abs2, [2; 3; 4])
29
```

Note the important difference between `sum(A)` and `reduce(+, A)` for arrays with small integer eltype:

```jldoctest
julia> sum(Int8[100, 28])
128

julia> reduce(+, Int8[100, 28])
-128
```

In the former case, the integers are widened to system word size and therefore the result is 128. In the latter case, no such widening happens and integer overflow results in -128.

---

```
sum(itr)
```

Returns the sum of all elements in a collection.

The return type is `Int` for signed integers of less than system word size, and `UInt` for unsigned integers of less than system word size.  For all other arguments, a common return type is found to which all arguments are promoted.

# Examples

```jldoctest
julia> sum(1:20)
210
```

---

```
sum(A::AbstractArray; dims)
```

Sum elements of an array over the given dimensions.

# Examples

```jldoctest
julia> A = [1 2; 3 4]
2×2 Array{Int64,2}:
 1  2
 3  4

julia> sum(A, dims=1)
1×2 Array{Int64,2}:
 4  6

julia> sum(A, dims=2)
2×1 Array{Int64,2}:
 3
 7
```


In [177]:
sum(a,2)

MethodError: MethodError: objects of type Array{Float64,1} are not callable
Use square brackets [] for indexing an Array.

In [187]:
a = rand(5)

5-element Array{Float64,1}:
 0.8309937075326719 
 0.18858530864517986
 0.7126934783837136 
 0.933182766211085  
 0.2298265218994795 

In [204]:
a = rand(2,2)
?norm

LoadError: syntax: invalid identifier name "?"

In [206]:
 ?opnorm

search: opnorm



```
opnorm(A::AbstractMatrix, p::Real=2)
```

Compute the operator norm (or matrix norm) induced by the vector `p`-norm, where valid values of `p` are `1`, `2`, or `Inf`. (Note that for sparse matrices, `p=2` is currently not implemented.) Use [`norm`](@ref) to compute the Frobenius norm.

When `p=1`, the operator norm is the maximum absolute column sum of `A`:

$$
\|A\|_1 = \max_{1 ≤ j ≤ n} \sum_{i=1}^m | a_{ij} |
$$

with $a_{ij}$ the entries of $A$, and $m$ and $n$ its dimensions.

When `p=2`, the operator norm is the spectral norm, equal to the largest singular value of `A`.

When `p=Inf`, the operator norm is the maximum absolute row sum of `A`:

$$
\|A\|_\infty = \max_{1 ≤ i ≤ m} \sum _{j=1}^n | a_{ij} |
$$

# Examples

```jldoctest
julia> A = [1 -2 -3; 2 3 -1]
2×3 Array{Int64,2}:
 1  -2  -3
 2   3  -1

julia> opnorm(A, Inf)
6.0

julia> opnorm(A, 1)
5.0
```

---

```
opnorm(x::Number, p::Real=2)
```

For numbers, return $\left( |x|^p \right)^{1/p}$. This is equivalent to [`norm`](@ref).

---

```
opnorm(A::Adjoint{<:Any,<:AbstracVector}, q::Real=2)
opnorm(A::Transpose{<:Any,<:AbstracVector}, q::Real=2)
```

For Adjoint/Transpose-wrapped vectors, return the operator $q$-norm of `A`, which is equivalent to the `p`-norm with value `p = q/(q-1)`. They coincide at `p = q = 2`. Use [`norm`](@ref) to compute the `p` norm of `A` as a vector.

The difference in norm between a vector space and its dual arises to preserve the relationship between duality and the dot product, and the result is consistent with the operator `p`-norm of a `1 × n` matrix.

# Examples

```jldoctest
julia> v = [1; im];

julia> vc = v';

julia> opnorm(vc, 1)
1.0

julia> norm(vc, 1)
2.0

julia> norm(v, 1)
2.0

julia> opnorm(vc, 2)
1.4142135623730951

julia> norm(vc, 2)
1.4142135623730951

julia> norm(v, 2)
1.4142135623730951

julia> opnorm(vc, Inf)
2.0

julia> norm(vc, Inf)
1.0

julia> norm(v, Inf)
1.0
```
